In [168]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [169]:
import numpy as np
from tqdm import trange, tqdm, tqdm_notebook
import scipy.io as sio
from kalman_estimation import Selector, get_mat_data

- matlab 代码
```matlab
f_name = [data_root, 'FROLS_', int2str(ndim), flag, '_est100_', sprintf('%2.2f', err_var), '.mat'];
save(f_name, 'coef_est100', 'terms_chosen100', 'ERRs100', 'mean_coef', 'var_coef');
```

In [170]:
con_terms_linear5 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)', 'x1(t-3)', 'x1(t-2)', 'x4(t-1)', 'x5(t-1)', 'x4(t-1)', 'x5(t-1)']  # 9
con_terms_nonlinear5 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)*x1(t-2)', 'x1(t-3)', 'x1(t-2)*x1(t-2)', 'x4(t-1)', 'x5(t-1)', 'x4(t-1)', 'x5(t-1)']  # 9
true_coefs5 = [0.95*np.sqrt(2), -0.9025, 0.5, -0.4, -0.5, 0.25*np.sqrt(2), 0.25*np.sqrt(2), -0.25*np.sqrt(2), 0.25*np.sqrt(2)]  # 9
con_terms_linear10 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)', 'x2(t-3)', 'x1(t-2)', 'x4(t-4)', 'x9(t-2)', 'x4(t-4)', 'x1(t-1)', 'x1(t-2)', 'x7(t-2)', 
                      'x8(t-3)', 'x9(t-3)', 'x8(t-3)', 'x9(t-3)', 'x7(t-4)']  # 16
con_terms_nonlinear10 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)*x1(t-10)', 'x2(t-3)', 'x1(t-2)', 'x4(t-4)', 'x9(t-2)', 'x4(t-4)', 'x1(t-1)*x1(t-10)', 'x1(t-2)', 'x7(t-2)', 
                      'x8(t-3)', 'x9(t-3)', 'x8(t-3)', 'x9(t-3)', 'x7(t-4)']  # 16
true_coefs10 = [0.95*np.sqrt(2), -0.9025, 0.5, 0.9, -0.5, 0.8, -0.4, -0.8, 0.4, -0.4, -0.9, 0.4, 0.3, -0.3, 0.4, -0.75]  # 16
noises = np.linspace(0.5, 4, 8)
con_terms5 = [2, 1, 1, 3, 2]
con_terms10 = [2, 1, 1, 1, 2, 1, 2, 3, 2, 1]
root = '../data/'

In [171]:
def corr_term(y_coef, terms_set, Kalman_S_No, var_name: str = 'x', step_name: str = 't'):
    n_dim, n_term = y_coef.shape
    func_repr = []
    for var in range(n_dim):
        y = {}
        for term in range(n_term):
            y[terms_set[Kalman_S_No[var, term]]] = y_coef[var, term]
        func_repr.append(y)
    return func_repr

In [172]:
def frols(noise_var, ndim, dtype, terms, length, root='../data/', trials=100):
    assert dtype in ['linear', 'nonlinear'], 'type not support!'
    terms_path = root + f'FROLS_{ndim}{dtype}_est100_{noise_var:2.2f}.mat'
    terms_pathx = root + f'{dtype}_terms{ndim}D_0.50trial1.mat'
    term = Selector(terms_pathx)
    candidate_terms = term.make_terms()
    y_coef = get_mat_data(terms_path, 'coef_est100')
    Kalman_S_No = get_mat_data(terms_path, 'terms_chosen100')
    flatten_coef = []
    for trial in trange(trials):
        ax, t, S_No = [], 0, Kalman_S_No[trial] - 1
        for i in range(ndim):
            terms_set = corr_term(y_coef[trial], candidate_terms, S_No)
            tmp = []
            for k in terms[t:t+length[i]]:
                tmp.append(terms_set[i][k] if k in terms_set[i] else np.nan)
            ax.extend(tmp)
            t += length[i]
        flatten_coef.append(ax)
    return np.stack(flatten_coef)

In [173]:
terms_pathx = root + f'linear_terms5D_0.50trial1.mat'
term = Selector(terms_pathx)
candidate_terms = term.make_terms()

In [174]:
term_dict = {t:i for i, t in enumerate(candidate_terms)}

In [175]:
coef = frols(2.5, 5, 'nonlinear', con_terms_nonlinear5, con_terms5)
coef

100%|███████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 7713.52it/s]


array([[ 1.31557986, -0.88518076,  0.74124799, -0.79235296, -0.47404816,
                nan,         nan, -0.76006489,  0.35023822],
       [ 1.33418651, -0.89692255,  0.73824   , -0.76350299, -0.53338268,
                nan,         nan, -0.76063927,  0.34056954],
       [ 1.32426818, -0.89459653,  0.74882781, -0.79738756, -0.54697314,
                nan,         nan, -0.75414867,  0.36159713],
       [ 1.3080536 , -0.86700576,  0.71948818, -0.78115476,         nan,
                nan,         nan, -0.75766548,  0.34849785],
       [ 1.34892001, -0.91423945,  0.61487514, -0.80702252,         nan,
                nan,         nan, -0.7608137 ,  0.34675958],
       [ 1.35542663, -0.89943803,  0.75538095, -0.77655311, -0.61284565,
         0.96078885,         nan, -0.76030335,  0.35427496],
       [ 1.33793796, -0.90283251,  0.60727555, -0.8093805 , -0.20337764,
                nan,         nan, -0.75299949,  0.34883427],
       [ 1.35968107, -0.9116894 ,  0.74653164, -0.80606151, -0

In [176]:
coef.shape

(100, 9)

In [177]:
coef.var(0)

array([4.00500496e-04, 3.47552794e-04, 3.23525712e-03, 4.25576901e-04,
                  nan,            nan,            nan, 7.69126593e-05,
       6.75310035e-05])

In [178]:
coef.mean(0)

array([ 1.34256891, -0.90381247,  0.70182456, -0.79324103,         nan,
               nan,         nan, -0.75718615,  0.35192632])